# 要約 
このJupyterノートブックは、「LMSYS Keras Gemma 2B」というタイトルで、LMSYS - Chatbot Arenaのコンペティションにおけるチャットボット応答の評価問題に取り組んでいます。具体的には、与えられたユーザーのプロンプトに対して複数の応答モデルから勝者を予測するためのモデルを構築およびトレーニングしています。

### 実装手法:
1. **ライブラリと環境の設定:**
   - `keras-nlp`および`tensorflow-text`を使用して自然言語処理のためのKeras機能を強化し、`tensorflow-cpu`をインストールしてTPUに悪影響を与えないよう設定しています。
   - JAXライブラリを利用し、高速な数値計算が可能な環境を整えています。

2. **デバイスの管理:**
   - KerasのディストリビューションAPIを使用して、TPUリソースを効率的に利用するためのデバイスメッシュを設定し、モデルの重みを8つのTPUに分散させています。

3. **データの前処理:**
   - 入力プロンプトとレスポンスのペアをCSVファイルから読み込み、適切に整形して新しいデータフレームを作成しています。このデータには、応答の勝利モデル情報も含まれています。
   - テキスト内の非表示文字（サロゲートペア）を取り除く関数を定義し、データクレンジングを行っています。

4. **モデルの構築:**
   - Kerasの`GemmaCausalLM`モデルを利用して、因果モデルをベースとした構造を設計しています。そして、独自の全結合層を追加して、最終的な出力を3クラスのsoftmax出力にセットしています。
   - LoRA（Low-Rank Adaptation）を使用してモデルの知識を効率的に適応させ、計算資源の無駄を減らしています。

5. **モデルのコンパイルとトレーニング:**
   - AdamWオプティマイザを利用し、学習率と重みの減衰を設定してモデルをコンパイルしています。
   - TensorFlowのデータセットAPIを利用してトレーニングデータを準備し、モデルを1エポック訓練しています。

6. **重みの保存:**
   - トレーニング後、LoRAで適応したモデルの重みを保存して、後で再利用できるようにしています。

このノートブックは、KerasとTensorFlowを利用して、複数の応答モデル間の優劣を予測するための効率的な機械学習モデルを構築するための枠組みを提供しています。

---


# 用語概説 
以下に、Jupyter Notebookの内容に関連する機械学習・深層学習の専門用語の簡易解説を列挙します。特に初心者がつまずきそうなマイナーなものやドメイン特有のものに焦点を当てています。

1. **TPU (Tensor Processing Unit)**:
   - Googleが開発した専用のハードウェアで、大規模な機械学習のモデルを効率的にトレーニングするために設計されています。TPUは特に行列演算を高速化するための最適化がされており、深層学習モデルのトレーニングに利用されます。

2. **JAX**:
   - Googleが開発した数値計算ライブラリで、NumPyに似たインターフェースを持ちながら、自動微分、GPU/TPUのサポート、高速な数値計算が可能です。研究者やデータサイエンティストが特に利用します。

3. **XLA (Accelerated Linear Algebra)**:
   - TensorFlowの背後にある最適化される線形代数コンパイラで、数値計算の効率を向上させることを目的としています。特に計算グラフを最適化し、さまざまなハードウェアで実行速度を向上させます。

4. **デバイスメッシュ (Device Mesh)**:
   - 分散処理のために、複数のデバイス（CPUまたはTPU）を論理的に構築した構成です。これにより、大規模モデルのトレーニングや推論が効率的に行えます。

5. **レイアウトマップ (Layout Map)**:
   - モデルの重みや計算を特定のデバイスに分散させるために使用される構造です。特にTPUのような特定のハードウェアでの効率的な計算を実現するために重要です。

6. **サロゲートペア (Surrogate Pairs)**:
   - Unicodeでの文字エンコーディングにおいて、2つの16ビットのコード単位（サロゲートペア）を使って1つの文字を表現する方法です。通常の文字としては表現できない範囲の文字に使用され、テキスト処理で問題を引き起こすことがあるため、除去が必要になる場合があります。

7. **LoRA (Low-Rank Adaptation)**:
   - 大規模な事前学習済みモデルの特定のタスク向けの微調整を行うための手法で、モデルのパラメータ数を増やさずに adaptationを行います。このテクニックは計算リソースを効率的に使用し、モデルの性能を向上させることができます。

8. **グローバル平均プール (Global Average Pooling)**:
   - 多次元のテンソル（通常は空間的な特徴マップ）のすべての要素の平均を取る操作です。画像分類や自然言語処理のタスクで、各特徴チャネルから1つの数値を抽出し、全体の特徴を要約するのに使われます。

9. **tf.data.Dataset**:
   - TensorFlowでデータを効率的に管理し、モデルに供給するためのAPIです。これを使うことで、データの前処理、シャッフル、バッチ処理などを簡便に行うことができます。

10. **ハイパーパラメータ (Hyperparameter)**:
    - モデルのトレーニングプロセスや構造を設定するためのパラメータであり、学習率やバッチサイズなどが含まれます。これらは学習前に設定され、モデルの性能に大きく影響します。

これらの解説が、ノートブックの内容を理解する助けになることを願っています。

---


# LMSYS Keras Gemma 2B

In [ ]:
!pip install -q -U keras-nlp tensorflow-text
# keras-nlp と tensorflow-text をアップグレードしてインストールします。
# これにより、NLP用のKerasライブラリとテキスト処理用のTensorFlowライブラリが使用可能になります。

!pip install -q -U tensorflow-cpu
# tensorflow-cpu をインストールします。
# これにより、TensorFlow がTPUにアクセスしようとしないようにします。
# TPU は特別なハードウェアで、通常のCPUの代わりに使われることがあるため、ここではCPU版を指定します。

In [ ]:
import jax

# JAXライブラリをインポートします。
# JAXは、高速な数値計算や自動微分を提供するライブラリです。

jax.devices()
# 使用可能なデバイスを表示します。
# この関数は、JAXが使用できるCPUやGPU、TPUなどのデバイスをリストします。

In [ ]:
import os

# Keras 3のディストリビューションAPIは、現時点ではJAXバックエンドでのみ実装されています。
os.environ["KERAS_BACKEND"] = "jax"
# TPUのメモリを事前に確保して、メモリの断片化と割り当てのオーバーヘッドを最小化します。
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.0"
# XLA（Accelerated Linear Algebra）Pythonクライアントのメモリ使用率を1.0に設定し、
# すべてのTPUメモリを確保します。これにより、メモリの効率的な使用が促進されます。

In [ ]:
import keras
import keras_nlp

# Kerasライブラリをインポートします。
# Kerasは、深層学習モデルを簡単に構築・トレーニングできる高水準なAPIです。

import keras_nlp
# keras_nlpライブラリをインポートします。
# keras_nlpは、自然言語処理（NLP）タスクに特化したKerasの拡張機能であり、
# モデルの構築やデータ処理を支援するための豊富なツールやモジュールを提供します。

In [ ]:
# (1, 8) の形状のデバイスメッシュを作成し、すべての8つのTPUに重みを分散させます。
device_mesh = keras.distribution.DeviceMesh(
    (8, 1),  # デバイスメッシュの形状を指定します。ここでは8つのTPUを使います。
    ["batch", "model"],  # バッチ次元とモデル次元を指定します。
    devices=keras.distribution.list_devices(),  # 使用可能なデバイスのリストを取得します。
) 
# このデバイスメッシュは、TPUのリソースを効果的に利用するために、
# モデルの重みを複数のTPUにまたがって分散させるために使用されます。

In [ ]:
model_dim = "model"

layout_map = keras.distribution.LayoutMap(device_mesh)

# 'token_embedding/embeddings'に一致する重みは、8つのTPUに分割されます。
layout_map["token_embedding/embeddings"] = (model_dim, None)

# Attentionレイヤー内のクエリ、キー、値の行列に対して一致する正規表現
layout_map["decoder_block.*attention.*(query|key|value)/kernel"] = (model_dim, None, None)

# Attention出力のカーネルに対するレイアウトマップの設定
layout_map["decoder_block.*attention_output/kernel"] = (model_dim, None, None)

# フィードフォワードゲーティングのカーネルに対するレイアウトマップの設定
layout_map["decoder_block.*ffw_gating.*/kernel"] = (None, model_dim)

# フィードフォワード線形変換のカーネルに対するレイアウトマップの設定
layout_map["decoder_block.*ffw_linear/kernel"] = (model_dim, None)

# このレイアウトマップは、モデルの重みがTPU間で適切に分散されるように配置を設定します。
# 各重みに対して適切なデバイスを指定することで、計算効率を向上させます。

In [ ]:
def remove_surrogates(text):
    # テキスト内のサロゲートペア（Unicodeの範囲0xD800から0xDFFF）を削除する関数です。
    return ''.join(char for char in text if not (0xD800 <= ord(char) <= 0xDFFF))
    # 上記の条件に基づいて、サロゲートペアではない文字のみを結合して新しい文字列を作成し、
    # サロゲートペアを排除したテキストを返します。

In [ ]:
from pandas import read_csv, DataFrame

input_columns = ['prompt', 'response_a', 'response_b']
label_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']

# CSVファイルから生のトレーニングデータセットを読み込みます。
raw_train_dataset = read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')

# nullでない場合にevalを使って最初の要素を取得し、入力カラムに適用します。
raw_train_dataset[input_columns] = raw_train_dataset[input_columns].map(lambda x: eval(x)[0] if 'null' not in x else None)

# 欠損値を持つ行を削除し、不要なカラム（'model_a', 'model_b'）を削除してインデックスをリセットします。
raw_train_dataset = raw_train_dataset.dropna().drop(['model_a', 'model_b'], axis=1).reset_index(drop=True)

# 新しいデータフレームを作成し、テキストとラベルを設定します。
train_dataset = DataFrame({
    'text' : raw_train_dataset[input_columns].apply(lambda x: '<start_of_turn>user\nFind which one is the best answer for the question:\n'+x['prompt']+'\n\nA:\n'+x['response_a']+'\n\nB\n:'+x['response_b']+'\n\nC:\n both right (or) both wrong<end_of_turn>\n<start_of_turn>model\n', axis=1).apply(lambda x: remove_surrogates(x)),
    # テキスト列には、ユーザーからのプロンプトとモデルの応答が含まれます。
    'label' : raw_train_dataset[label_columns].apply(lambda x: x.values.tolist(), axis=1)
    # ラベル列には、各応答の勝者をリストとして格納します。
    # 'label' : raw_train_dataset[label_columns].apply(lambda x: 'A' if x.values.tolist()[0] == 1 else 'B' if x.values.tolist()[1] == 1 else 'C', axis=1)
})

# データセットを最初の4000行に制限します。
train_dataset = train_dataset[:4000]
raw_train_dataset = raw_train_dataset[:4000]

In [ ]:
len(train_dataset)
# train_datasetのデータフレームの行数を取得します。
# この行数はトレーニングデータセットに含まれるサンプルの数を示します。

In [ ]:
model_parallel = keras.distribution.ModelParallel(
    layout_map=layout_map,  # 先に定義したレイアウトマップを使用します。
    batch_dim_name="batch",  # バッチ次元の名前を指定します。
)

# モデルの並列処理の設定を行います。
keras.distribution.set_distribution(model_parallel)
# これにより、モデルがTPUでのトレーニングのために適切に分散されるようになります。
# 同時に複数のTPUを利用してトレーニングが行えるようになります。

In [ ]:
keras.config.set_floatx("float16")
# Kerasの浮動小数点数の精度をfloat16に設定します。
# これにより、メモリ使用量が削減され、高速な計算が可能になります。

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("/kaggle/input/gemma/keras/gemma_instruct_2b_en/2")
# GemmaCausalLMモデルを指定されたプレセットから読み込みます。
# このモデルは因果言語モデルであり、自然言語の生成タスクに使用されます。

gemma_lm.summary()
# モデルの概要を表示します。
# これにより、モデルの構造、パラメータの数、レイヤーの情報などが確認できます。

In [ ]:
gemma_lm.backbone.enable_lora(rank=8)
# GemmaモデルのバックボーンにLoRA（Low-Rank Adaptation）を有効にします。
# rank=8は、LoRAのランクを8に設定することを意味します。
# LoRAは、モデルのパラメータ数を増やさずに、特定のタスクに対するモデルの適応能力を向上させるための手法です。
# これにより、少ない計算リソースで効果的にモデルを微調整できます。

In [ ]:
# gemma_lm.backbone.layers[:16]の各レイヤーについて、学習可能な状態を無効化します。
# for layer in gemma_lm.backbone.layers[:16]:
#     layer.trainable = False
# 最初の16層の重みを固定し、トレーニング中にこれらのパラメータが更新されないようにします。
# これにより、モデル全体のトレーニング時間を短縮し、過学習を防ぐことができます。

In [ ]:
gemma_lm.summary()
# モデルの概要を再表示します。
# これにより、モデルの構造、レイヤーの情報、トレーニング可能なパラメータの数などが確認でき、
# 変更後のモデルの状態を確認できます。

In [ ]:
def preprocess_fn(text, label=None):
    # テキストとラベルを前処理する関数です。
    preprocessed = gemma_lm._preprocessor(text, sequence_length=3072)[0]
    # Gemmaモデルのプリプロセッサを使用してテキストを3072のシーケンス長に前処理します。
    
    # 前処理関数が必要な入力のみを返すようにします。
    return (
        {'token_ids': preprocessed['token_ids'], 'padding_mask': preprocessed['padding_mask']}, 
        label if label is not None else {'token_ids': preprocessed['token_ids'], 'padding_mask': preprocessed['padding_mask']}
    )
    # token_idsとpadding_maskからなる辞書を返します。
    # ラベルがNoneでない場合は、ラベルも一緒に返します。そうでなければ、token_idsとpadding_maskの辞書を返します。

In [ ]:
gemma_lm.layers[-1]
# Gemmaモデルの最後のレイヤーを取得します。
# これにより、モデルの出力層や最後の層の詳細を確認できます。
# モデルのアーキテクチャを理解するために役立ちます。

In [ ]:
import gc

# Gemmaモデルの最後のレイヤーを削除します。
del gemma_lm.layers[-1]

# ガーベジコレクションを実行して、不要なメモリを解放します。
gc.collect()
# これにより、削除されたレイヤーに関連するメモリが解放され、
# メモリ使用量を最適化します。

In [ ]:
import tensorflow as tf
from keras.layers import Input, Dense, Flatten, GlobalAveragePooling1D
from keras import Model

# モデルの入力を定義します。
inputs = {
    "token_ids": keras.Input(shape=(3072,), dtype=tf.int32, name="token_ids"),
    "padding_mask": keras.Input(shape=(3072,), dtype=tf.int32, name="padding_mask"),
}

# Gemmaモデルのバックボーンを通して入力を処理します。
x = gemma_lm.backbone(inputs)
print(x.shape)  # Gemmaモデルの出力の形状を表示します。

# 出力をグローバル平均プールします。
x = GlobalAveragePooling1D()(x)
print(x.shape)  # プーリング後の形状を表示します。

# 最後に、3クラスのsoftmax出力を持つ全結合層を定義します。
outputs = Dense(3, 'softmax')(x)
# 入力と出力を使ってモデルを構築します。
model = Model(inputs, outputs)

In [ ]:
optimizer = keras.optimizers.AdamW(
                learning_rate=5e-5,  # 学習率を5e-5に設定します。
                weight_decay=0.01,   # 重みの減衰を0.01に設定します。
)

# 重み減衰を適用しない変数の名前を指定します。
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])
# これにより、バイアスとスケールに関連するパラメータには重み減衰が適用されません。
# これは、過学習を防ぎつつモデルの学習パフォーマンスを向上させるために行われます。

In [ ]:
model.compile(optimizer, loss=tf.keras.losses.CategoricalCrossentropy(),)
# モデルをコンパイルします。
# 指定したオプティマイザ（optimizer）を使用し、損失関数としてカテゴリカルクロスエントロピーを設定します。
# カテゴリカルクロスエントロピーは、多クラス分類の問題でよく使用される損失関数であり、
# モデルの出力と教師データのクラスラベルとの間の不一致を測定します。

In [ ]:
import tensorflow as tf

# トレーニングデータセットからテキストとラベルを含むデータセットを作成します。
ds = tf.data.Dataset.from_tensor_slices((train_dataset.text.values, raw_train_dataset[label_columns].values))\
    .batch(8)  # バッチサイズを8に設定します。
    
# 前処理関数をデータセットに適用します。
ds = ds.map(preprocess_fn)

# データセットをシャッフルします。
ds = ds.shuffle(ds.cardinality())
# cardinality()はデータセットのサンプル数を返します。
# シャッフルにより、トレーニングデータの順序がランダム化され、モデルの学習が改善されることが期待されます。

In [ ]:
train_split = ds.take(int(len(ds) * 0.9))
# データセットの90%を訓練用分割として取得します。

val_split = ds.skip(int(len(ds) * 0.9)).take(int(len(ds) * 0.1))
# データセットの残りの10%を検証用分割として取得します。
# skip()で90%をスキップし、take()で次の10%を取得します。

# モデルを訓練します。
histories = model.fit(train_split, validation_data=[val_split], epochs=1, batch_size=8)
# 訓練を1エポック行い、バッチサイズを8に設定します。
# validation_data引数を用いて、検証用データを設定し、モデルの性能を評価します。

In [ ]:
model.get_layer("gemma_backbone").save_lora_weights('/kaggle/working/lora19.lora.h5')
# Gemmaモデルのバックボーンレイヤーに対してLoRA（Low-Rank Adaptation）重みを保存します。
# 指定されたパス（'/kaggle/working/lora19.lora.h5'）に重みをHDF5形式で保存します。
# これにより、後でこの重みを再利用したり、試験したりすることができます。